# Time evolution and ground states with matrix product states (MPS)

In this notebook, we define a nearest-neighbour Hamiltonian on a finite, linear chain. We compute time evolution under the Hamiltonian and the ground state of the Hamiltonian.

Refer e.g. to https://arxiv.org/abs/1008.3477 for an introduction and for original references describing the algorithms used below.

Related keywords:

- density matrix renormalization group (DMRG)

- time-dependent density matrix renormalization group (t-DMRG)

- time-evolving block decimation (TEBD)

- Variational matrix product state (MPS) algorithms

- Tensor trains (TT)

The code below uses Python and `mpnum` (http://mpnum.readthedocs.io/). This notebook is part of https://github.com/milan-hl/mpnum-examples/.

Below, very basic algorithms are used. Refer to the literature for better available algorithms.

## Definitions

Open ``init.ipynb`` to see which MPSs and MPOs it defines.

In [1]:
%run init.ipynb

In [2]:
# Use qubits - local dimension 2
ldim = 2
# Run on 6 qubits
n_sites = 6
width = 2  # nearest-neighbour
# Parameter of the Hamiltonian
h = .3
# Evolution time
t = 0.9
steps = 5
tau = t / steps

Define the following nearest-neighbour Hamiltonian of $n$ qubits:

\begin{align}
H &= \sum_{i=0}^{n-2} h_{i,i+1} \\
h_{i, i+1} &= \sigma_x^i \sigma_x^{i+1} + \sigma_y^i \sigma_y^{i+1} + h \sigma_z^i +  \delta_{i,n-1} h \sigma_z^{i+1}
\end{align}

In [3]:
h_term = mp.chain((mpx, mpx)) + mp.chain((mpy, mpy)) + h * mp.chain((mpz, mid))
h_terms = [h_term.copy() for _ in range(n_sites - 1)]
h_terms[-1] += h * mp.chain((mid, mpz))

Numerical compression (without modifying the Hamiltonian terms) does not reduce the bond dimension:

In [4]:
print([t.ranks for t in h_terms])
[t.compress(method='svd', relerr=1e-10) for t in h_terms]
print([t.ranks for t in h_terms])

[(3,), (3,), (3,), (3,), (4,)]
[(3,), (3,), (3,), (3,), (4,)]


## Time evolution with sparse matrices

In [5]:
# Convert local terms (MPOs on two qubits) to full matrices on two qubits
h_terms_arr = [t.to_array_global().reshape([ldim**width] * 2) for t in h_terms]

In [6]:
H = 0

for pos, term in enumerate(h_terms_arr):
    H += spa.kron(
        spa.eye(ldim**pos), 
        spa.kron(
            term,
            spa.eye(ldim**(n_sites - width - pos))
        )
    )

In [7]:
psi_0 = mp.chain(([mup, mdown] * ((n_sites + 1) // 2))[:n_sites])
psi_0_arr = psi_0.to_array().ravel()

In [8]:
psi_t_spa = spa.linalg.expm_multiply(-1j * H * t, psi_0_arr)
print('Norm of the evolved state:', np.linalg.norm(psi_t_spa))

Norm of the evolved state: 1.0


Algorithm used by `expm_multiply()` is referenced at https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.expm_multiply.html

## Time evolution with MPS

We do a first order trotter approximation:
$$
\exp(-i H t) \approx [ \exp(-i H_o \tau) \exp(-i H_e \tau) ]^M, \quad \tau = t / M
$$
where
$$
H = H_e + H_o, \quad
H_e = \sum_{i=0}^{n/2} h_{2i,2i+1}, \quad
H_o = \sum_{i=0}^{n/2} h_{2i+1, 2i+2}
$$

In [9]:
h_evos = [
    # Convert results from matrix to MPO (on 2 qubits)
    mp.MPArray.from_array_global(
        # Compute exp(-i h_{i,i+1} t) as full matrix (on 2 qubits)
        sp.linalg.expm(
            (-1j * tau)
            # Convert MPO on 2 qubits to square matrix
            * term.to_array_global().reshape([ldim**width] * 2)
        )
        .reshape([ldim] * (2 * width)),
        ndims=2
    )
    for term in h_terms
]

In [10]:
h_evo_even = mp.chain(h_evos[0::2])
h_evo_odd = mp.chain(h_evos[1::2])

In [11]:
psi = psi_0
for i in range(steps):
    psi = mp.partialdot(h_evo_even, psi, 0)
    psi = mp.partialdot(h_evo_odd, psi, 1)
    psi.compress(method='svd', relerr=1e-10)
    
psi_t_arr = psi.to_array().ravel()
print('Norm of the time-evolved state:', np.linalg.norm(psi_t_arr))
print('Overlap with solution from sparse matrix:', abs(np.vdot(psi_t_arr, psi_t_spa)))

Norm of the time-evolved state: 1.0
Overlap with solution from sparse matrix: 0.959732582153


Notes:
- Only 5 Trotter steps cause small overlap

- For larger numbers of sites, a larger compression error needs to be tolerated (otherwise, bond dimension and memory usage will grow)

## Ground state with sparse matrices

Compute the eigenvalue which has the smallest real part and a corresponding eigenvector:

In [12]:
eigval_spa, eigvec_spa = spa.linalg.eigsh(H, k=1, which='SR')

Algorithm used by `eigsh()` is referenced at https://docs.scipy.org/doc/scipy/reference/generated/generated/scipy.sparse.linalg.eigsh.html

## Ground state with matrix products states (MPS) and operators (MPO)

The following code performs a variational search for eigenvectors of a matrix product operator (MPO) Hamiltonian whose eigenvalue has a small real part, with the search restricted to matrix product states (MPS) of a certain bond dimension.

In [13]:
# Convert local terms to MPO
H_mpo = mp.local_sum(h_terms)

# Look for the eigenvalue with smallest real part
import functools as ft
eigs_fun = ft.partial(spa.linalg.eigsh, k=1, which='SR')

eigval, eigvec = mp.linalg.eig(H_mpo, num_sweeps=5, startvec_rank=4, eigs=eigs_fun)
eigvec_arr = eigvec.to_array().ravel()

# Todo: Verify that discarded imaginary part is small
Hpsi = mp.dot(H_mpo, eigvec)
ept_Hsq = mp.norm(Hpsi)**2
ept_H = mp.inner(eigvec, Hpsi).real
rel_var = (ept_Hsq - ept_H**2) / ept_H**2

print('Eigenvalue difference:', eigval - eigval_spa)
print('Eigenvector overlap:', abs(np.vdot(eigvec_spa, eigvec_arr)))
print('Relative variance:', rel_var)

Eigenvalue difference: [ 0.01046594]
Eigenvector overlap: 0.99901663645
Relative variance: 0.00126286092335


Notes:

- **No guarantee** that the ground state has been found
- Number of iterations (`num_sweeps`) must be chosen be the user
- For possible convergence criteria, refer to the literature (e.g. https://arxiv.org/abs/1008.3477)
- To verify whether the result is an eigenvector, compute e.g. $(\langle H^2 \rangle - \langle H \rangle^2) / \langle H \rangle^2$
- Larger bond dimensions (`startvec_rank`) enables more precise results
- Documentation is at http://mpnum.readthedocs.io/en/latest/mpnum.html#mpnum.linalg.eig